Create a new repository on your Github account and name it Coursera_Capstone. I have prepared a guide to walk you through the process of creating a repository and setting it up. For Mac users, click here. For Windows users, click here.

Now, start a Jupyter Notebook using any platform that you are comfortable with and do the following:

Write some markdown to explain that this notebook will be mainly used for the capstone project.
Import the pandas library as pd.
Import the Numpy library as np.
Print the following the statement: Hello Capstone Project Course!
Push the Notebook to your Github repository and submit a link to the notebook on your Github repository.

In [2]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!
